In [1]:
import cx_Oracle
import pandas as pd
import sqlalchemy
from sqlalchemy.exc import SQLAlchemyError

In [2]:
#url="https://dkube-examples-data.s3.us-west-2.amazonaws.com/monitoring-insurance/training-data/insurance.csv" #insurance dataset
url="https://dkube-examples-data.s3.us-west-2.amazonaws.com/monitoring-titanic/training-data/titanic.csv"
name = "titanic"
df = pd.read_csv(url)
print(df)

     PassengerId  Survived  Pclass     Sex   Age  SibSp  Parch     Fare  \
0              1         0       3    male  22.0      1      0   7.2500   
1              2         1       1  female  38.0      1      0  71.2833   
2              3         1       3  female  26.0      0      0   7.9250   
3              4         1       1  female  35.0      1      0  53.1000   
4              5         0       3    male  35.0      0      0   8.0500   
..           ...       ...     ...     ...   ...    ...    ...      ...   
886          887         0       2    male  27.0      0      0  13.0000   
887          888         1       1  female  19.0      0      0  30.0000   
888          889         0       3  female   NaN      1      2  23.4500   
889          890         1       1    male  26.0      0      0  30.0000   
890          891         0       3    male  32.0      0      0   7.7500   

                         timestamp  
0       2021-11-09 14:15:23.337560  
1    2021-11-09 14:21:51.

Verify and change the column types , no need to change the object or timestamp datatype

In [3]:
df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Sex             object
Age            float64
SibSp            int64
Parch            int64
Fare           float64
timestamp       object
dtype: object

In [4]:
df['timestamp'] = pd.to_datetime(df.timestamp)

In [5]:
df.dtypes

PassengerId             int64
Survived                int64
Pclass                  int64
Sex                    object
Age                   float64
SibSp                   int64
Parch                   int64
Fare                  float64
timestamp      datetime64[ns]
dtype: object

In [6]:
df

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,timestamp
0,1,0,3,male,22.0,1,0,7.2500,2021-11-09 14:15:23.337560000
1,2,1,1,female,38.0,1,0,71.2833,2021-11-09 14:21:51.652166743
2,3,1,3,female,26.0,0,0,7.9250,2021-11-09 14:28:19.966773487
3,4,1,1,female,35.0,1,0,53.1000,2021-11-09 14:34:48.281380231
4,5,0,3,male,35.0,0,0,8.0500,2021-11-09 14:41:16.595986975
...,...,...,...,...,...,...,...,...,...
886,887,0,2,male,27.0,0,0,13.0000,2021-11-13 13:49:30.079135024
887,888,1,1,female,19.0,0,0,30.0000,2021-11-13 13:55:58.393741768
888,889,0,3,female,NaN,1,2,23.4500,2021-11-13 14:02:26.708348512
889,890,1,1,male,26.0,0,0,30.0000,2021-11-13 14:08:55.022955256


Push df to oracle

from tqdm import tqdm
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

def insert_with_progress(df, engine, name):
    conn = engine.connect()
    chunksize = int(len(df) / 10)
    with tqdm(total=len(df)) as pbar:
        for i, cdf in enumerate(chunker(df, chunksize)):
            replace = "replace" if i == 0 else "append"
            cdf.to_sql(name=name, con=conn, if_exists=replace, index=False) 
            pbar.update(chunksize)
            tqdm._instances.clear()

In [7]:
DIALECT = 'oracle'
SQL_DRIVER = 'cx_oracle'
USERNAME = 'system' #enter your username
PASSWORD = 'oracle' #enter your password
HOST = '192.168.200.117' #enter the oracle db host url
PORT = 49161 # enter the oracle port number
SERVICE = 'XE' # enter the oracle db service name
ENGINE_PATH_WIN_AUTH = DIALECT + '+' + SQL_DRIVER + '://' + USERNAME + ':' + PASSWORD +'@' + HOST + ':' + str(PORT) + '/?service_name=' + SERVICE

engine = sqlalchemy.create_engine(ENGINE_PATH_WIN_AUTH)

In [8]:
from sqlalchemy.dialects.oracle import TIMESTAMP
#convert all object type to varchar since they get very slow
dtyp = {c:sqlalchemy.types.VARCHAR(df[c].str.len().max()) for c in df.columns[df.dtypes == 'object'].tolist()}
dtyp['timestamp'] = TIMESTAMP
df.to_sql(name=name,con=engine, index=False,dtype=dtyp)

In [9]:
dtyp

{'Sex': VARCHAR(length=6), 'timestamp': sqlalchemy.sql.sqltypes.TIMESTAMP}